In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_curve, auc, precision_score, recall_score
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import LocalOutlierFactor
from numpy import arange
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report

In [2]:
benign_df = pd.read_csv(r'D:\Kuliah\ProyekAkhir\New Data\resample\Benign_resample.csv')
benign_df.head(2)

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,15.854422,-0.343690,-0.086569,0.096351,0.142000,0.002723,0.706619,1.982467,1.798333,-0.034669,...,-0.059807,-0.095769,-0.109896,-0.112787,-0.020891,-0.339324,-0.282883,-0.341153,-0.315336,1
1,-0.060352,5.598377,2.253048,4.681794,3.829148,0.125465,3.569438,-0.039001,2.481753,5.125594,...,-0.059807,1.103561,0.162506,0.515708,1.673959,6.184780,12.942534,8.511326,2.479016,1


In [3]:
ldap_df = pd.read_csv(r'D:\Kuliah\ProyekAkhir\New Data\resample\LDAP_resample_4000.csv')
ldap_df

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.045263,-0.015250,-0.005277,-0.005954,0.289320,-0.002643,0.322770,0.323287,0.323284,-0.016174,...,0.226640,-0.007737,-0.005256,-0.007436,-0.00637,-0.013105,-0.005207,-0.013185,-0.012838,-1
1,0.045263,-0.015250,-0.005277,-0.005954,0.309328,-0.002643,0.344531,0.345012,0.345025,-0.016174,...,0.226640,-0.007737,-0.005256,-0.007436,-0.00637,-0.013105,-0.005207,-0.013185,-0.012838,-1
2,0.045263,-0.015250,-0.005277,-0.005954,0.309328,-0.002643,0.344531,0.345012,0.345025,-0.016174,...,0.226640,-0.007737,-0.005256,-0.007436,-0.00637,-0.013105,-0.005207,-0.013185,-0.012838,-1
3,0.045263,-0.015249,-0.005277,-0.005954,0.309328,-0.002643,0.344531,0.345012,0.345025,-0.016174,...,0.226640,-0.007737,-0.005256,-0.007436,-0.00637,-0.013105,-0.005207,-0.013185,-0.012838,-1
4,0.045263,-0.015250,-0.005277,-0.005954,0.289320,-0.002643,0.322770,0.323287,0.323284,-0.016174,...,0.226646,-0.007737,-0.005256,-0.007436,-0.00637,-0.013105,-0.005207,-0.013185,-0.012838,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3720,0.045263,-0.015250,-0.005277,-0.005954,0.309328,-0.002643,0.344531,0.345012,0.345025,-0.016174,...,0.226640,-0.007737,-0.005256,-0.007436,-0.00637,-0.013105,-0.005207,-0.013185,-0.012838,-1
3721,0.045263,-0.015250,-0.005277,-0.005954,0.309328,-0.002643,0.344531,0.345012,0.345025,-0.016174,...,0.226640,-0.007737,-0.005256,-0.007436,-0.00637,-0.013105,-0.005207,-0.013185,-0.012838,-1
3722,0.045263,-0.015250,-0.005277,-0.005954,0.309328,-0.002643,0.344531,0.345012,0.345025,-0.016174,...,0.226640,-0.007737,-0.005256,-0.007436,-0.00637,-0.013105,-0.005207,-0.013185,-0.012838,-1
3723,0.045263,-0.015250,-0.005277,-0.005954,0.309328,-0.002643,0.344531,0.345012,0.345025,-0.016174,...,0.226640,-0.007737,-0.005256,-0.007436,-0.00637,-0.013105,-0.005207,-0.013185,-0.012838,-1


In [4]:
benign_test_df = pd.read_csv(r'D:\Kuliah\ProyekAkhir\New Data\resample\Benign_resample_4000.csv')

In [5]:
# novelty detection
df_test = pd.concat([benign_test_df, ldap_df], ignore_index=True)
df_train = benign_df

In [6]:
df_test = df_test[[' Protocol', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Flow Packets/s', ' Fwd IAT Std', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd Packets/s', ' Packet Length Mean', ' SYN Flag Count', ' RST Flag Count', ' URG Flag Count', ' CWE Flag Count', ' Down/Up Ratio', ' Avg Fwd Segment Size', ' min_seg_size_forward', 'Active Mean', ' Label']]
df_train = df_train[[' Protocol', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Flow Packets/s', ' Fwd IAT Std', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd Packets/s', ' Packet Length Mean', ' SYN Flag Count', ' RST Flag Count', ' URG Flag Count', ' CWE Flag Count', ' Down/Up Ratio', ' Avg Fwd Segment Size', ' min_seg_size_forward', 'Active Mean', ' Label']]

In [7]:
# training_data
X_train = df_train.drop([' Label'], axis=1)
y_train = df_train[' Label']
# testing_data
X_test = df_test.drop([' Label'], axis=1)
y_test = df_test[' Label']

In [8]:
# Novelty detection with 8000 sample balanced
for a in range(1, 3):
    for i in arange(0.1, 0.5, 0.1):
        clf = LocalOutlierFactor(n_neighbors=a, novelty=True, contamination=i)
        clf.fit(X_train)
        y_pred_test = clf.predict(X_test)
        print('Neighbour : ' + str(a))
        print('contamination : ' + str(i))
        print('---Classification Report---')
        print(classification_report(y_test, y_pred_test))

Neighbour : 1
contamination : 0.1
---Classification Report---
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00      3725
           1       1.00      1.00      1.00      4003

    accuracy                           1.00      7728
   macro avg       1.00      1.00      1.00      7728
weighted avg       1.00      1.00      1.00      7728

Neighbour : 1
contamination : 0.2
---Classification Report---
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00      3725
           1       1.00      1.00      1.00      4003

    accuracy                           1.00      7728
   macro avg       1.00      1.00      1.00      7728
weighted avg       1.00      1.00      1.00      7728

Neighbour : 1
contamination : 0.30000000000000004
---Classification Report---
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00      3725
           1       1.00      1.00  